In [1]:
import numpy as np
import pandas as pd
from datasets import ClassLabel, Dataset, DatasetDict, Features, Value, Sequence
import pyarrow as pa
import numpy as np
from tqdm.notebook import tqdm
import torch ## version >= 1.8.2
import torch.nn as nn
import pytorch_lightning as pl ## version == 1.4.9
import datasets ## version == 2.1.0
from transformers import AutoTokenizer, AutoModel ## version == 4.12.3

/home/aift-ml/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-03 14:16:45.460512: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 14:16:45.590238: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-03 14:16:46.106719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: N

데이터 로드

In [2]:
train_df = pd.read_excel("/home/aift-ml/workspace/lm/seoul_faq_train/finetune/data/seoul/seoul_faq_0503_train.xlsx")

In [3]:
dev_df = pd.read_excel("/home/aift-ml/workspace/lm/seoul_faq_train/finetune/data/seoul/seoul_faq_0503_dev.xlsx")

In [4]:
total_label_list = list()
for i in train_df.values:
    sentence = i[0]
    label = str(i[1])
    label_kor = str(i[2])
    if label_kor not in total_label_list:
        total_label_list.append(label_kor)
        
print(len(total_label_list))


437


In [5]:
class_labels = total_label_list

In [6]:
train_sentence= list()
train_labels = list()
for i in train_df.values:
    sentence = i[0]
    class_label = str(i[2])
    label = [class_labels.index(class_label)]
    train_sentence.append(sentence)
    train_labels.append(label)


랜덤하게 질문 합치는 복합 질문 생성

In [7]:
import random

In [8]:
index_list = list()
for idx in range(len(train_sentence)):
    index_list.append(idx)


In [12]:
sum_train_sentence = []
sum_train_labels = []

for idx in range(len(train_sentence)):
    indexs = random.choices(index_list,k=2)
    if train_labels[indexs[0]] != train_labels[indexs[1]]:
        new_sentence = f"{train_sentence[indexs[0]]} {train_sentence[indexs[1]]}"
        new_label = [train_labels[indexs[0]][0], train_labels[indexs[1]][0]]
        sum_train_sentence.append(new_sentence)
        sum_train_labels.append(new_label)
sum_train_sentence.extend(train_sentence)
sum_train_labels.extend(train_labels)

In [13]:
dev_index_list = list()
dev_sentence= list()
dev_labels = list()
for idx, i in enumerate(dev_df.values):
    dev_index_list.append(idx)
    sentence = i[0]
    class_label = str(i[2])
    label = [class_labels.index(class_label)]
    dev_sentence.append(sentence)
    dev_labels.append(label)

In [14]:
sum_dev_sentence = []
sum_dev_labels = []

for idx in range(len(dev_sentence)):
    indexs = random.choices(dev_index_list,k=2)
    if dev_labels[indexs[0]] != dev_labels[indexs[1]]:
        new_sentence = f"{dev_sentence[indexs[0]]} {dev_sentence[indexs[1]]}"
        new_label = [dev_labels[indexs[0]][0], dev_labels[indexs[1]][0]]
        sum_dev_sentence.append(new_sentence)
        sum_dev_labels.append(new_label)
        
sum_dev_sentence.extend(dev_sentence)
sum_dev_labels.extend(dev_labels)

dataset 으로 만들기

In [15]:
class_label = ClassLabel(names=total_label_list)

In [16]:
train_data = {
    'ID': [str(i) for i in range(len(sum_train_sentence))],
    'text': [s for s in sum_train_sentence],
    'labels': [l for l in sum_train_labels]
}

In [17]:
test_data  = {
    'ID': [i for i in range(len(sum_dev_sentence))],
    'text': [s for s in sum_dev_sentence],
    'labels': [l for l in sum_dev_labels]
}

In [18]:
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)
validation_df = pd.DataFrame(test_data)

In [19]:
features = Features({
    'ID': Value(dtype='string', id=None),
    'text': Value(dtype='string', id=None),
    'labels': Sequence(feature=class_label)
})

In [20]:
train_dataset = Dataset.from_pandas(train_df, features=features)
test_dataset = Dataset.from_pandas(test_df, features=features)
validation_dataset = Dataset.from_pandas(validation_df, features=features)

In [22]:
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': validation_dataset
})

In [23]:
## convert the integer labels into multi-hot form (44-dimensional).

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform(dataset["train"]["labels"])
test_labels = mlb.fit_transform(dataset["test"]["labels"])
val_labels = mlb.fit_transform(dataset["validation"]["labels"])

print("train_labels shape ::: {}".format(train_labels.shape))
print("test_labels shape :::: {}".format(test_labels.shape))
print("val_labels shape ::::: {}".format(val_labels.shape))
print("\ncool..!!")

train_labels shape ::: (25026, 437)
test_labels shape :::: (5239, 437)
val_labels shape ::::: (5239, 437)

cool..!!


In [24]:
## extract the texts, since we will use a custom datset not the huggingface dataset.

train_texts = dataset["train"]["text"]
test_texts = dataset["test"]["text"]
val_texts = dataset["validation"]["text"]

In [25]:
## and the label names...

LABELS = dataset["train"].features["labels"].feature.names

tokenization

In [26]:
## download the pretrained tokenizer from huggingface.

MODEL_NAME = "klue/roberta-base" 
# MODEL_NAME = 'beomi/KcELECTRA-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [27]:
def token_masking(encoding, prob):
    for i, token in enumerate(encoding["input_ids"][0]):
        if token not in [0,1,2,3]: # 0 ~ 3, [PAD], [UNK], [CLS], and [SEP], respectively.
            if np.random.uniform(0,1) < prob:
                encoding["input_ids"][0][i] = 4 # 4 is [MASK]
                
    return encoding

In [28]:
def token_switching(encoding, prob):
    for i, token in enumerate(encoding["input_ids"][0]):
        if token not in [0,1,2,3,4]: # 0 ~ 4, [PAD], [UNK], [CLS], [SEP], and [MASK], respectively.
            if np.random.uniform(0,1) < prob:
                encoding["input_ids"][0][i] = np.random.choice(np.arange(5,tokenizer.vocab_size), 1)[0]
                
    return encoding

In [29]:
def mask_and_switch(encoding, prob=0.1):
    encoding = token_masking(encoding, prob/2)
    encoding = token_switching(encoding, prob/2)
    
    return encoding

custom dataset

In [30]:
from torch.utils.data import Dataset

In [31]:
## maximum token lengths

MAX_LENGTH = 128

In [32]:
## define our dataset...!

class KOTEDataset(Dataset):
    
    def __init__(self, texts, labels, tokenizer, max_length:int=MAX_LENGTH,
                would_you_like_some_mask_and_switch:bool=False):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.mask = would_you_like_some_mask_and_switch
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx:int):
        text = self.texts[idx]
        labels = self.labels[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            return_token_type_ids=False,
        )
        
        if self.mask:
            encoding = mask_and_switch(encoding, prob=0.1)
        else:
            pass
        
        return dict(
          input_ids=encoding["input_ids"].flatten(),
          attention_mask=encoding["attention_mask"].flatten(),
          labels=torch.FloatTensor(labels), ## must be a float tensor.
        )

In [33]:
## create the datasets.

train_dataset = KOTEDataset(train_texts, train_labels, tokenizer=tokenizer, would_you_like_some_mask_and_switch=True)
test_dataset = KOTEDataset(test_texts, test_labels, tokenizer=tokenizer)
val_dataset = KOTEDataset(val_texts, val_labels, tokenizer=tokenizer)

modeling

In [34]:
## download the pretrained electra model.

electra= AutoModel.from_pretrained(MODEL_NAME, return_dict=True)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for

In [35]:
electra.config

RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.21.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32000
}

dataloader with pl

In [36]:
from torch.utils.data import DataLoader

In [37]:
class KOTEDataModule(pl.LightningDataModule):
    
    def __init__(self, train_dataset, test_dataset, val_dataset, batch_size=16):
        super().__init__()
        self.batch_size = batch_size
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.val_dataset = val_dataset

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=6, ## choose a befitting number depending on your environment.
        )
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=6, ## choose a befitting number depending on your environment.
        )
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            num_workers=6, ## choose a befitting number depending on your environment.
        )

In [38]:
BATCH_SIZE = 16 ## about 28 ~ 30 Gb memory required, if my memory serves me right.

data_module = KOTEDataModule(
  train_dataset,
  test_dataset,
  val_dataset,
  batch_size=BATCH_SIZE,
)

model

In [39]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [40]:
INITIAL_LR = 1e-4

In [41]:
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None, size_average=True, device='cpu'):
        super(FocalLoss, self).__init__()
        """
        gamma(int) : focusing parameter.
        alpha(list) : alpha-balanced term.
        size_average(bool) : whether to apply reduction to the output.
        """
        self.gamma = gamma
        self.alpha = alpha
        self.size_average = size_average
        self.device = device

    def forward(self, input, target):
        # input : N * C (btach_size, num_class)
        # target : N (batch_size)

        CE = F.cross_entropy(input, target, reduction='none')  # -log(pt)
        pt = torch.exp(-CE)  # pt
        loss = (1 - pt) ** self.gamma * CE  # -(1-pt)^rlog(pt)

        if self.alpha is not None:
            alpha = torch.tensor(self.alpha, dtype=torch.float).to(self.device)
            # in case that a minority class is not selected when mini-batch sampling
            if len(self.alpha) != len(torch.unique(target)):
                temp = torch.zeros(len(self.alpha)).to(self.device)
                temp[torch.unique(target)] = alpha.index_select(0, torch.unique(target))
                alpha_t = temp.gather(0, target)
                loss = alpha_t * loss
            else:
                alpha_t = alpha.gather(0, target)
                loss = alpha_t * loss

        if self.size_average:
            loss = torch.mean(loss)

        return loss

In [42]:
class KOTETagger(pl.LightningModule):
    
    def __init__(self, n_training_steps=None, n_warmup_steps=None, gamma_for_expLR=None):
        super().__init__()
        self.electra = electra
        self.classifier = nn.Linear(self.electra.config.hidden_size, 437) ## the label dimension == 44 <-- what an ominous number for asians though... <-- I didn't intend it!
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        
        ## the loss
        # self.criterion = nn.BCELoss()
        # self.criterion = nn.CrossEntropyLoss()
        self.criterion = FocalLoss()
        
    def forward(self, input_ids, attention_mask, labels=None):
        output = self.electra(input_ids, attention_mask=attention_mask)
        output = output.last_hidden_state[:,0,:] ## [CLS] of the last hidden state
        output = self.classifier(output)
        output = torch.sigmoid(output)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)
        
        torch.cuda.empty_cache()
        
        return loss, output
    
    def step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self.forward(input_ids, attention_mask, labels)

        preds = outputs

        y_true = list(labels.detach().cpu())
        y_pred = list(preds.detach().cpu())

        return {"loss": loss, "y_true": y_true, "y_pred": y_pred}
    
    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx)

    def validation_step(self, batch, batch_idx):
        return self.step(batch, batch_idx)
    
    def epoch_end(self, outputs, state="train"):
        loss = torch.tensor(0, dtype=torch.float)
        for out in outputs:
            loss += out["loss"].detach().cpu()
        loss = loss / len(outputs)

        y_true = []
        y_pred = []
        for out in outputs:
            y_true += out["y_true"]
            y_pred += out["y_pred"]

        self.log(state + "_loss", float(loss), on_epoch=True, prog_bar=True)
        print(f"[Epoch {self.trainer.current_epoch} {state.upper()}] Loss: {loss}")
        return {"loss": loss}
    
    def training_epoch_end(self, outputs):
        self.epoch_end(outputs, state="train")

    def validation_epoch_end(self, outputs):
        self.epoch_end(outputs, state="val")
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=INITIAL_LR)
        
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.n_warmup_steps,
            num_training_steps=self.n_training_steps
        )
        
        return dict(
          optimizer=optimizer,
          lr_scheduler=dict(
            scheduler=scheduler,
            interval="step"
          )
        )

In [43]:
## determine the schedule for our optimizer

N_EPOCHS = 20

steps_per_epoch = len(train_dataset) // BATCH_SIZE
TOTAL_STEPS = steps_per_epoch * N_EPOCHS
WARMUP_STEPS = TOTAL_STEPS // 5
WARMUP_STEPS, TOTAL_STEPS

(6256, 31280)

In [44]:
## define the model.

model = KOTETagger(
    n_warmup_steps=WARMUP_STEPS,
    n_training_steps=TOTAL_STEPS,
)

training

In [45]:
## set a logger and some stuffs...

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

## the check point
checkpoint_callback = ModelCheckpoint(
    dirpath="./multi_output_roberta_focal_3",
    filename="epoch{epoch}-val_loss{val_loss:.4f}",
    monitor="val_loss",
    save_top_k=3,
    mode="min",
    auto_insert_metric_name=False,
)

## for early stopping
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=5, min_delta=0.00)

## the logger
logger = TensorBoardLogger("./multi_output_roberta_focal_3", name="multi_output_roberta_focal_3")

In [46]:
## trainer

N_EPOCHS = 25 ## redefine the number of the epochs, just to make sure there is no more room to improve.

trainer = pl.Trainer(
    logger=logger,
    callbacks=[checkpoint_callback, early_stopping_callback],
    max_epochs=N_EPOCHS,
    gpus=[0], ## GPU number
    progress_bar_refresh_rate=5
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [47]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [48]:
## about 4 ~ 5 hours to reach the optimum...
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/aift-ml/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name       | Type         | Params
--------------------------------------------
0 | electra    | RobertaModel | 110 M 
1 | classifier | Linear       | 336 K 
2 | criterion  | FocalLoss    | 0     
--------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
443.817   Total estimated model params size (MB)


Epoch 6:  10%|█         | 195/1893 [00:20<02:53,  9.79it/s, loss=7.85, v_num=0, val_loss=7.640, train_loss=7.670]

prediction

In [73]:
from glob import glob

par_dir = '/home/aift-ml/workspace/lm/seoul_faq_train/finetune/multi_output_roberta_focal_2/'
best_ckpt = sorted(glob(par_dir + '*.ckpt'))[-1]
best_ckpt

'/home/aift-ml/workspace/lm/seoul_faq_train/finetune/multi_output_roberta_focal_2/epoch21-val_loss10.2073.ckpt'

In [74]:
gruesome_mind_reader = KOTETagger.load_from_checkpoint(best_ckpt)

In [75]:
gruesome_mind_reader.eval()
gruesome_mind_reader.freeze()

In [93]:
def multi_model_inference(text):
  THRESHOLD = 0.6
  encoding = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=128,
    return_token_type_ids=False,
    padding="max_length",
    return_attention_mask=True,
    return_tensors="pt",
  )

  _, predictions = gruesome_mind_reader(encoding["input_ids"], encoding["attention_mask"])
  predictions = predictions.flatten().numpy()
  results = list()
  for l,p in zip(LABELS, predictions):
      if p < THRESHOLD:
          continue
      results.append((l,p))
  results.sort(key=lambda x:x[1], reverse=True)
  for idx, r in enumerate(results):
    print(f"{idx} {r[0]}: {r[1]}")

In [94]:
multi_model_inference("사망자 수는 몇 명인가? 그리고 초등학교는 몇 개??")

0 사망자수: 0.9998960494995117
1 한국교육학술정보원_학교알리미 학교정보: 0.9998233914375305
2 서울시 학교 기본정보: 0.8325072526931763
3 사망원인별 사망자수: 0.787848711013794


In [95]:
multi_model_inference("고용률 알려줄 수 있나요.. 그리고 최근 실업률도요")

0 성/연령별 실업률: 0.9999929666519165
1 성/연령별 고용률: 0.9989653825759888


In [96]:
multi_model_inference("백혈병으로 죽는 사람 수와 사망률 궁금")

0 사망원인별 사망자수: 0.9999836683273315
1 사망원인별 사망률: 0.9999371767044067


In [97]:
multi_model_inference("현재 유치원 학급 수와 유치원생 신입 원아수가 얼마나 되는지 궁금")

0 유치원 신입원아 수: 0.9999632835388184
1 유치원 학급수: 0.998214602470398


In [98]:
multi_model_inference("마포구에 전기차 급속충전하는 곳과 전동킥보드 견인 현황은 어디서 보나요")

0 서울시 전동킥보드 견인 현황: 0.9996391534805298
1 서울시 전기차 급속충전기 정보 현황: 0.9996383190155029


In [99]:
multi_model_inference("정류장별로 환승하는 사람 몇 명인지와 제설함 어디에 있는지 알고 싶어요")

0 서울시 정류장별 사용자유형별 환승 승객수 및 환승시간 정보: 0.9999268054962158
1 서울시 제설함 위치정보: 0.9994366765022278


In [100]:
multi_model_inference("노인 인구 몇 명이야?")

0 고령인구: 0.9999990463256836
1 외국인고령자: 0.9999812841415405
2 내국인고령자: 0.9985413551330566
3 고령인구 구성비: 0.9748948216438293
4 서울시 노인(65세이상) 고용지표 통계: 0.9335790276527405
5 주민등록인구: 0.7977495789527893
6 성/연령별 인구: 0.6776670217514038


In [102]:
multi_model_inference("서울시 노인인구 수와 초등학교 수 그리고 버스정류장 수가 궁금합니다")

0 한국교통안전공단_버스정보시스템(BIS)_버스정류소정보: 0.9984763264656067
1 고령인구: 0.9920511841773987
2 외국인고령자: 0.9908663630485535
3 서울시 버스정류소 위치정보: 0.9883832335472107
4 한국교육학술정보원_학교알리미 학교정보: 0.9519776105880737
5 내국인고령자: 0.8971028923988342


In [106]:
multi_model_inference("대학생은 또 몇 명인지 알려줘요")

0 성/연령/교육정도별 인구: 0.9999997615814209
1 한국교육학술정보원_학교알리미 학교정보: 0.9999719858169556
